In [10]:
import os 
import json
import cv2
import numpy as np
import clip
import torch
from PIL import Image
import pathlib
from pathlib import Path
import glob
import argparse
import jsonlines
import json
import time
import deeplake as dl
import tqdm

In [15]:
WHISPER_RESULTS_DATASET_PATH = f'/mnt/storage_ssd/whisper_results_parallel_15'
ds = dl.load(WHISPER_RESULTS_DATASET_PATH)
ds.summary()

/mnt/storage_ssd/whisper_results_parallel_15 loaded successfully.
Dataset(path='/mnt/storage_ssd/whisper_results_parallel_15', tensors=['caption', 'segment_metadata', 'video_filename', 'video_filepath'])

      tensor        htype     shape      dtype  compression
     -------       -------   -------    -------  ------- 
     caption        text    (17213, 1)    str     None   
 segment_metadata   json    (16748, 1)    str      lz4   
  video_filename    text    (17213, 1)    str     None   
  video_filepath    text    (17213, 1)    str     None   


In [18]:
import pprint
for sample in ds:
  pprint.pprint(sample.segment_metadata.data()['value'])
  break

{'end': 5.017125,
 'segment_index': 0,
 'segment_word_list': [{'end': 1.3043749999999998,
                        'start': 1.184,
                        'word': 'HEY'},
                       {'end': 1.6255, 'start': 1.3245, 'word': 'GUYS'},
                       {'end': 1.7860624999999999,
                        'start': 1.7058125,
                        'word': "IT'S"},
                       {'end': 2.026875, 'start': 1.806125, 'word': 'THUG'},
                       {'end': 2.56875, 'start': 2.0670625, 'word': 'SNIPER'},
                       {'end': 2.8697500000000002,
                        'start': 2.749375,
                        'word': 'AND'},
                       {'end': 2.93, 'start': 2.889875, 'word': 'ME'},
                       {'end': 3.7929375, 'start': 3.73275, 'word': 'AND'},
                       {'end': 3.9535, 'start': 3.833125, 'word': 'MY'},
                       {'end': 4.2344375, 'start': 3.9735625, 'word': 'BUDDY'},
                       {'end': 

In [3]:
OLD_DS_PATH = '/mnt/storage_ssd/v0_for_whisper_p15'
DATASET_PATH        = '/mnt/storage_ssd/v0_for_whisper_parallel_15'
BASE_DIR            = '/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/'
# BASE_DIR            = '/mnt/storage_ssd/'
BATCH_NAME          = 'parallel_15'
REMOTE_WHISPER_FILE = f'{BASE_DIR}/{BATCH_NAME}_whisper_output.jsonl'
REMOTE_CLIP_DIR     = f'{BASE_DIR}/{BATCH_NAME}_clip_output'
REMOTE_SCENE_FILE   = f'{BASE_DIR}/{BATCH_NAME}_scene_output.jsonl'

In [15]:
def main():
  ''' Create dataset '''
  if False and os.path.exists(DATASET_PATH):
    ds = dl.load(DATASET_PATH)
    print(ds.summary())
    all_stems = ds.video_stem.data()['value']
    already_completed_stems = set(all_stems)
    print("already completed: ", len(already_completed_stems))
  else:
    print(f"Creating/overwriting new dataset, sleeping 3 seconds to allow you to cancel.\nAt Path: {DATASET_PATH}")  
    # time.sleep(3)
    ds = dl.empty(DATASET_PATH, overwrite=True)
    with ds:
      # ds.create_tensor('video_metadata', htype='json', sample_compression='lz4') # compression: https://docs.deeplake.ai/en/latest/Compressions.html
      ds.create_tensor('video_name', htype='text', dtype=str, sample_compression=None)
      ds.create_tensor('video_filepath', htype='text', dtype=str, sample_compression=None)
      # ds.create_tensor('segment_frames', htype='image', dtype=np.uint8, sample_compression='jpg')
      # ds.create_tensor('text_caption_embeddings', htype='image', dtype=np.float16, sample_compression='lz4') # CLIP produces FP16 embeddings.
      # ds.create_tensor('frame_embeddings', htype='image', dtype=np.float16, sample_compression='lz4') # compression: https://docs.deeplake.ai/en/latest/Compressions.html

main()

Creating/overwriting new dataset, sleeping 3 seconds to allow you to cancel.
At Path: /mnt/storage_ssd/v0_for_whisper_p15


In [16]:
def add_raw_videos_to_dataset():
  print(f"Globbing paths from {os.path.join(BASE_DIR, BATCH_NAME, '*')}")
  raw_video_paths = glob.glob(os.path.join(BASE_DIR, BATCH_NAME, '*'), recursive = True)
  ds = dl.load(DATASET_PATH)
  print("Adding to dataset")
  counter = 0
  with ds:
    for video_path in tqdm.tqdm(raw_video_paths):
      video_name = str(pathlib.Path(video_path).name)
      # no groups 
      ds.append({
        'video_name': video_name,
        'video_filepath': video_path,
      })
      # one group per video
      # ds.create_group(video_name)
      # ds[video_name].create_tensor('video_metadata', htype='json', sample_compression='lz4') # compression: https://docs.deeplake.ai/en/latest/Compressions.html
      # ds[video_name].create_tensor('video_name', htype='text', dtype=str, sample_compression=None)
      # ds[video_name].append({
      #   'video_name': video_name,
      #   'video_metadata': metadata,
      # })
      # ds[video_name].video_name.append(video_name)
      # ds[video_name].video_metadata.append(metadata)
      # if counter > 100:
        # break
      counter += 1

add_raw_videos_to_dataset()

Globbing paths from /mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_15/*
/mnt/storage_ssd/v0_for_whisper_p15 loaded successfully.
Adding to dataset


100%|██████████| 25000/25000 [00:20<00:00, 1229.82it/s]


In [74]:

dl.deepcopy(OLD_DS_PATH, DATASET_PATH)

Copying dataset: 100%|██████████| 21/21 [00:00<00:00

/mnt/storage_ssd/v0_for_whisper_parallel_15 loaded successfully.


Dataset(path='/mnt/storage_ssd/v0_for_whisper_parallel_15', tensors=['video_filepath', 'video_name'])

In [4]:
l = f'/mnt/storage_ssd/whisper_results_{BATCH_NAME}'
ds = dl.load(l)
ds.summary()

/mnt/storage_ssd/whisper_results_parallel_15 loaded successfully.
Dataset(path='/mnt/storage_ssd/whisper_results_parallel_15', tensors=['caption', 'segment_metadata', 'video_filename', 'video_filepath'])

      tensor        htype    shape    dtype  compression
     -------       -------  -------  -------  ------- 
     caption        text    (5, 1)     str     None   
 segment_metadata   json    (1, 1)     str      lz4   
  video_filename    text    (5, 1)     str     None   
  video_filepath    text    (5, 1)     str     None   


In [78]:
paths = ds.video_filepath.data()['value']
paths[0]


'/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_15/jCciuZa8yXI_The Fumble_38323_Kevin Durant CAUGHT Hitting On Hottie At OKC Game & FINALLY Opens Up About BURNER Accounts!.webm'

In [32]:

ds = dl.load(f'/mnt/storage_ssd/whisper_results_{BATCH_NAME}')
ds.summary()
completed_videos = set()
# Get all completed vids for this batch iteratively (change to non-iter later)
for index, i in enumerate(ds):
  # print(index)
  # print(i.video_filename.data())
  # print(i.caption.data())
  # i.video_filename.data()['value']
  try:
    completed_videos.add(i.video_filepath.data()['value'])
  except:
    print(index)
# files = set(files) - completed_videos
# files = list(files)
len(completed_videos)



/mnt/storage_ssd/whisper_results_parallel_15 loaded successfully.
Dataset(path='/mnt/storage_ssd/whisper_results_parallel_15', tensors=['caption', 'segment_metadata', 'video_filename', 'video_filepath'])

      tensor        htype    shape    dtype  compression
     -------       -------  -------  -------  ------- 
     caption        text    (41, 1)    str     None   
 segment_metadata   json    (41, 1)    str      lz4   
  video_filename    text    (41, 1)    str     None   
  video_filepath    text    (41, 1)    str     None   


2

In [37]:
BATCH_NAME = 'parallel_15'
test_db = f'/mnt/storage_ssd/whisper_results_{BATCH_NAME}'
dl.delete(test_db)

In [24]:
ds = dl.load(DATASET_PATH)
# ds.summary()
files = ds.video_filepath.data()['value']

/mnt/storage_ssd/v0_for_whisper_parallel_15 loaded successfully.


In [26]:
len(set(files) - completed_videos)

25000

In [68]:
files = ds.video_filepath.data()['value']

In [43]:
len(set(files) - set(files[0:10]))

23292

In [37]:
files = [ str(file) for file in files if not str(file).endswith( ('.txt','.vtt', 'json') ) ]
print("After filtering -- Number of files:", len(files))

After filtering -- Number of files: 23302


In [36]:
dicts = [{"a": 0}, {"a": 1}]
num_dicts = len(dicts)
for dic in dicts:
  dic["b"] = num_dicts
dicts

[{'a': 0, 'b': 2}, {'a': 1, 'b': 2}]